# This notebook works with the Vprey project and it's current config.yaml file.

**Note**: It will also create the project if not already specified.

In [ ]:
import deeplabcut as dlc
import random
import shutil
import glob
import os

# From paths.py
import paths

In [ ]:
# Find all videos in the data directory
all_videos = glob.glob(paths.vrexperiment_path + os.sep + "*.avi")

## If project already created, run the snipped below

In [ ]:
# Get config path
config_path = paths.config_path_vr

# Read config file
cfg = dlc.auxiliaryfunctions.read_config(config_path)

# Get project directory
project_directory = cfg['project_path']

# Get list of local videos as found in the config
project_video_list = [vid for vid in cfg["video_sets"]]

# Convert to DLC 2.2 MultiAnimal
Only needs to be done once - DONE

In [ ]:
dlc.convert2_maDLC(config_path, userfeedback=True)

In [ ]:
## If you haven't created a project, run this cell

In [ ]:
dlc.create_new_project?

In [ ]:
# Randomly select 15 videos to train on
remote_video_list = []
for i in range(20):
    remote_video_list.append(random.choice(all_videos))

# Depreciated
# video_list = [r"06_30_2020_17_06_13_DG_200526_d_succ_lowFR.avi",
#               r"06_30_2020_15_06_59_DG_200526_b_succ_lowFR.avi",
#               r"06_23_2020_15_03_01_DG_200526_c_succ.avi",
#               r"06_30_2020_14_41_22_DG_200526_b_test_lowFR.avi",
#               r"06_26_2020_16_25_35_DG_200526_c_succ_lowFR.avi",
#               r"06_22_2020_12_24_42_DG_200526_b_succ.avi",
#               r"06_22_2020_11_25_50_DG_200526_a_test.avi",
#               r"03_12_2020_15_35_46_MM_200128_a_succ.avi",
#               r"03_05_2020_16_26_13_MM_200128_a_succ.avi", 
#               r"11_10_2019_23_46_06_DG_190417_a_fail.avi",
#               r"08_07_2019_18_05_49_DG_190416_a_succ.avi",
#               r"08_03_2019_17_34_12_DG_190416_a_succ.avi",
#               r"07_08_2020_16_10_58_DG_200526_b_succ_blackCr.avi",
#               r"07_08_2020_12_04_55_DG_200526_a_succ_blackCr.avi",
#               r"07_08_2020_11_43_42_DG_200526_a_test_blackCr.avi",
#               r"07_08_2020_16_09_13_DG_200526_b_test_nonrewarded_blackCr.avi",
#               r"07_08_2020_16_06_56_DG_200526_b_test_nonrewarded_blackCr.avi"
#              ]
# remote_video_list = [os.path.join(remote_data_directory, vid) for vid in video_list]

# Create the project if not already done
dlc.create_new_project("VPrey", "Matt", remote_video_list, working_directory=r"D:\dlc_vr_arena", copy_videos=True)

Make sure to add the path to the config file into ``paths.py``

In [ ]:
# Get config and path info
config_path = paths.config_path
cfg = dlc.auxiliaryfunctions.read_config(config_path)
project_directory = cfg['project_path']
project_video_list = [vid for vid in cfg["video_sets"]]

## Pick videos not contained in the training set for testing

In [ ]:
# Randomly pick videos to test the network that are not part of the training set and copy to the project directory
test_video_dir = os.path.join(project_directory, "test-videos")
if not os.path.isdir(test_video_dir):
    os.mkdir(test_video_dir)

    # Randomly pick five videos outside of the training set to thest the network on
    test_vids = []
    while(len(test_vids) < 5):
        vid = random.choice(all_videos)
        if vid not in project_video_list:
            dest = shutil.copy2(vid, test_video_dir)
            test_vids.append(dest)

else:
    test_vids = glob.glob(test_video_dir + os.sep + "*.avi")

## Add extra videos (if needed)

In [ ]:
# If adding new videos, do that here by randomly selecting some not in the original data set or the test set
extra_vids = []
while(len(extra_vids) < 5):
    vid = random.choice(all_videos)
    if vid not in project_video_list or test_vids:
        extra_vids.append(vid)
        
dlc.add_new_videos(config_path, extra_vids, copy_videos=True)

## Extract and label frames

In [ ]:
dlc.extract_frames?

In [ ]:
# extract frames from the videos
# will go through all of the videos on the list, so if many videos, it might take a while
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', crop="GUI", userfeedback=False);

In [ ]:
# label frames
# generates folders in the labeled-data folder with the video name
dlc.label_frames(config_path)

In [ ]:
# check the labels
# generates folders in the labeled-data folder with the video name but the actual labels printed on the pics
dlc.check_labels(config_path);

## Create dataset and train network

In [ ]:
dlc.create_training_dataset?

In [ ]:
# create the training set
# also creates the Pose.yaml in the dlc-models/train dataset, which contains the setting for training, i.e. take a look
dlc.create_training_dataset(config_path, augmenter_type='default');

In [ ]:
# train the network
dlc.train_network(config_path, max_snapshots_to_keep=10, displayiters=50000, saveiters=50000)

## Evaluate network

In [ ]:
# evaluate the network
dlc.evaluate_network(config_path, Shuffles=[1], plotting=True)

## Analyze new videos

In [ ]:
dlc.analyze_videos?

In [ ]:
dlc.analyze_videos(config_path, test_vids)

In [ ]:
dlc.create_labeled_video?

In [ ]:
# use the path defined above
dlc.create_labeled_video(config_path, test_vids)

In [ ]:
# extract outlier frames
dlc.extract_outlier_frames?

In [ ]:
# extract outlier frames
dlc.extract_outlier_frames(config_path, project_video_list, outlieralgorithm='uncerrtain', p_bound=0.8)
# deeplabcut.extract_outlier_frames?

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
# Merge the dataset, then move to check labels and retrain

dlc.merge_datasets(config_path)